# Paper Mockup
## Test the whole process, including fake human reviewers.
- Generate a corpus of 3 AnalysisRuns with 3 hypotheses each
- 6 Reviewers, 3 to impersonate human reviewers 
- ReviewPlan that applies the 6 reviewers to the 3 analysis runs
- Generate ReviewSets and execute
- For each Reviewer, gather the reviews and run create_ranking_vector
- Plot the Reviewers and the (fake) human reviewers
- Produce the heatmap comparing the reviewers
- [next - significance analysis]


### Set up parent directory and db connection

In [ ]:
import sys
import os

# Add the parent directory of the current script to the Python path
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
print(cwd)
print(dirname)
sys.path.append(dirname)

print(sys.path)

/Users/rudi/agent_evaluation/notebooks
/Users/rudi/agent_evaluation
['/Users/rudi/anaconda3/envs/agent_eval/lib/python311.zip', '/Users/rudi/anaconda3/envs/agent_eval/lib/python3.11', '/Users/rudi/anaconda3/envs/agent_eval/lib/python3.11/lib-dynload', '', '/Users/rudi/anaconda3/envs/agent_eval/lib/python3.11/site-packages', '/Users/rudi/agent_evaluation']


In [ ]:
from models.analysis_plan import AnalysisPlan
from services.analysisrunner import AnalysisRunner
from models.review_plan import ReviewPlan
from services.reviewrunner import ReviewRunner
from app.sqlite_database import SqliteDatabase
from app.config import load_database_config

# Load the db connection details
# db_type, uri, user, password = load_database_config(path='~/ae_config/test_config.ini')
# self.db = Database(uri, db_type, user, password)

_, database_uri, _, _ = load_database_config()
db = SqliteDatabase(database_uri)

## Generate AnalysisRuns

In [ ]:
analysis_plan_ids = [p1, p2, p3]
analysis_runs = []
for ap_id in analysis_plan_ids:
    analysis_plan = AnalysisPlan.load(db, ap_id)
    analysis_run = analysis_plan.generate_analysis_run()
    analysis_runs.append(analysis_run)

for analysis_run in analysis_runs:
    runner = AnalysisRunner(db, analysis_run.object_id)
    result = runner.run()



## Load the Reviewers

## Load the ReviewPlans, generate the ReviewSets

In [ ]:
review_plan_ids = ["review_plan_ad5c1fe8-dc76-4940-a938-bdfbd569f42d"]
review_sets = []
for review_plan_id in review_plan_ids:
    review_plan = ReviewPlan.load(db, review_plan_id)
    # Generate an empty ReviewSet from the ReviewPlan
    review_set = review_plan.generate_review_set()
    review_sets.append(review_set)

review_sets[0]

## Run the ReviewSets
Populate the ReviewSets with Reviews

In [ ]:
for review_set in review_sets:
    # Run the ReviewSet using a ReviewRunner
    runner = ReviewRunner(db, review_set.object_id)
    result = runner.run()

review_sets[0]

## Generate the Reviewer judgment vectors from the Reviews in the ReviewSets
The vectors correspond to the given ordered list of ReviewSets. They are not comparable otherwise.

We create a datastructure in which each Reviewer is associated with its judgment vector

In [ ]:
from app.analysis import create_review_judgment_vector, create_judgment_vector
from models.analyst import Analyst

reviewer_judgment_vectors = {}

# TODO: add error checking to be sure that the Reviewers match
# across the ReviewSets
for reviewer_id in review_sets[0].reviewer_ids:
    reviewer = Analyst.load(reviewer_id)
    vectors = []
    
    for review_set in review_sets:
        vector = create_review_judgment_vector(review_set, reviewer)
        vectors.append(vector)

    reviewer_judgment_vectors[reviewer_id] = create_judgment_vector(vectors)

    

reviewer_judgment_vectors
    


## Plot the judgment vectors
 - Label the points with the Reviewer names
 - The Reviewer names are prefixed with "H.", which is stripped off before labeling
 - Agents in red squares, Humans in blue circles.

In [ ]:
from app.analysis import visualize_judgment_vectors

visualize_judgment_vectors(reviewer_judgment_vectors)

## Create the Reviewer similarity heatmap
 - higher similarity is more red

In [ ]:
from app.analysis import reviewer_similarity_heatmap

reviewer_similarity_heatmap(reviewer_judgment_vectors)